Importing Required libraries

In [3]:
import math
import numpy as np
np.set_printoptions(suppress=True)

import pandas as pd

X_filename = "X.csv"
Y_filename = "y.csv"

Defining required functions for GCM

In [4]:
def distance(x, y, alpha):
    # Calculate psychological distance between memory examplar x and stimulus y
    dist = 0
    for i in range(len(x)):
        dist += (alpha[i] * (x[i] - y[i]))

    return abs(dist)

def similarity(dist, beta):
    # similary measure after calculating distance between x and y
    return math.exp(-beta * dist)

def get_exempler_set(dataframe, category_R):
    # Extract exemplar set dataframe of a particular label
    category_dataframe = dataframe[dataframe.label == category_R]
    return category_dataframe[["weight", 'height']]


def get_count(x, exemplar_set):
    # Calculate N(R, x)
    # count of the number of times x has been recorded as being in category R before
    values_df = exemplar_set.value_counts().reset_index()
    x_weight, x_height = x.weight, x.height
    row = values_df.loc[(values_df.weight == x_weight) & (values_df.height == x_height)]
    return row.iloc[0][0]

def exemplar_vote(y, dataframe, category_R, gamma_R, alpha, beta):
    # calculate P(R|y) without the normalization factor

    exemplar_set = get_exempler_set(dataframe, category_R)
    # print(exemplar_set)
    prob_R_y = 0 
    for x in exemplar_set.iterrows():
        x_ = x[1]
        N_R_x = get_count(x_, exemplar_set)
        distance_x_y = distance(x_, y, alpha=alpha)
        similarity_x_y = similarity(distance_x_y, beta=beta)
        prob_R_y += N_R_x * similarity_x_y
    
    prob_R_y *= gamma_R 

    return prob_R_y
    
def get_probablity(y, dataframe,categories = [1, 2, 3], alpha=[1,1], beta=1, gamma_R=[1,1,1]):
    # there are 3 category labels, 1, 2, 3,
    # we calculate P(R|y) for each category, then normalize it according to formula and 
    # return probablity for each label

    prob = []
    prob.append(exemplar_vote(y, dataframe, categories[0], gamma_R[0], alpha=alpha, beta=beta))
    prob.append(exemplar_vote(y, dataframe, categories[1], gamma_R[1], alpha=alpha, beta=beta))
    prob.append(exemplar_vote(y, dataframe, categories[2], gamma_R[2], alpha=alpha, beta=beta))

    return prob/sum(prob)

def get_output(X_df, Y_df, alpha=[1,1], beta=1, gamma_R=[1,1,1]):
    # given X_df and Y_df, calculate probablity and store output for each stimuli

    output_dict = {}
    for i in range(len(Y_df)):
        output_dict[str(Y_df.iloc[i].values)] = get_probablity(Y_df.iloc[i], X_df, alpha=alpha, beta=beta, gamma_R=gamma_R)
        # probablity.append(get_probablity(Y_df.iloc[i], X_df))

    return output_dict

In [5]:
X_df = pd.read_csv(X_filename, names=['weight', 'height', 'label'])
Y_df = pd.read_csv(Y_filename, names=['weight', 'height',])

In [6]:
Y_df.head()

weight  height
0      74      67
1      69      63
2      92      81
3      64      61
4      66      84

Visualizing probablity output

Case 1) When there is no bias

In [7]:
output_dict = get_output(X_df, Y_df)
for key in output_dict.keys():
    print(f"For stimuli {key}, the predicted label is {np.argmax(output_dict[key]) + 1} and probablities are {output_dict[key]}")

For stimuli [74 67], the predicted label is 2 and probablities are [0.         0.90465068 0.09534932]
For stimuli [69 63], the predicted label is 2 and probablities are [0.00000004 0.9999966  0.00000335]
For stimuli [92 81], the predicted label is 3 and probablities are [0.         0.00000113 0.99999887]
For stimuli [64 61], the predicted label is 2 and probablities are [0.00037445 0.99962553 0.00000002]
For stimuli [66 84], the predicted label is 3 and probablities are [0.         0.00010416 0.99989584]
For stimuli [76 68], the predicted label is 3 and probablities are [0.         0.07866294 0.92133706]
For stimuli [61 58], the predicted label is 1 and probablities are [0.9838622 0.0161378 0.       ]
For stimuli [64 76], the predicted label is 2 and probablities are [0.         0.97291818 0.02708182]
For stimuli [68 66], the predicted label is 2 and probablities are [0.00000001 0.99997427 0.00002572]
For stimuli [34 61], the predicted label is 1 and probablities are [1. 0. 0.]


Now according to question, these are the assumptions
1) he is polite, and is far more likely to call someone big average than
large
2) He is more likely to use weight than height to make category
judgments about size.

So we set the value of alpha=[2,1] means we give more weightage to weigth than height <br>
And we set value of gamma_R = [33, 40, 27] meaning label 1 gets 1/3rd weigtage, label 2 gets slightly higher weightage and label 3 low

In [8]:
output_dict = get_output(X_df, Y_df, alpha=[2,1], gamma_R=[33,40,27])
for key in output_dict.keys():
    print(f"For stimuli {key}, the predicted label is {np.argmax(output_dict[key]) + 1} and probablities are {output_dict[key]}")

For stimuli [74 67], the predicted label is 2 and probablities are [0.         0.90986859 0.09013141]
For stimuli [69 63], the predicted label is 2 and probablities are [0. 1. 0.]
For stimuli [92 81], the predicted label is 3 and probablities are [0. 0. 1.]
For stimuli [64 61], the predicted label is 2 and probablities are [0. 1. 0.]
For stimuli [66 84], the predicted label is 2 and probablities are [0.         0.57738105 0.42261895]
For stimuli [76 68], the predicted label is 3 and probablities are [0.        0.0004581 0.9995419]
For stimuli [61 58], the predicted label is 2 and probablities are [0.02064169 0.97935831 0.        ]
For stimuli [64 76], the predicted label is 2 and probablities are [0.         0.99999996 0.00000004]
For stimuli [68 66], the predicted label is 2 and probablities are [0. 1. 0.]
For stimuli [34 61], the predicted label is 1 and probablities are [1. 0. 0.]


We can see that for case, [66,84] normally output predicted is 3, but after adding bias, we are getting output =2

Also covering case where label 1 and label 2 bias are equal and label 3 has low bias

In [9]:
output_dict = get_output(X_df, Y_df, alpha=[2,1], gamma_R=[40,40,20])
for key in output_dict.keys():
    print(f"For stimuli {key}, the predicted label is {np.argmax(output_dict[key]) + 1} and probablities are {output_dict[key]}")

For stimuli [74 67], the predicted label is 2 and probablities are [0.         0.93163857 0.06836143]
For stimuli [69 63], the predicted label is 2 and probablities are [0. 1. 0.]
For stimuli [92 81], the predicted label is 3 and probablities are [0. 0. 1.]
For stimuli [64 61], the predicted label is 2 and probablities are [0. 1. 0.]
For stimuli [66 84], the predicted label is 2 and probablities are [0.         0.64842792 0.35157208]
For stimuli [76 68], the predicted label is 3 and probablities are [0.         0.00061833 0.99938167]
For stimuli [61 58], the predicted label is 2 and probablities are [0.02491115 0.97508885 0.        ]
For stimuli [64 76], the predicted label is 2 and probablities are [0.         0.99999997 0.00000003]
For stimuli [68 66], the predicted label is 2 and probablities are [0. 1. 0.]
For stimuli [34 61], the predicted label is 1 and probablities are [1. 0. 0.]


This assumption gives no significant change